In [2]:
from accelerate import Accelerator
from accelerate.logging import get_logger

accelerator = Accelerator()

logger = get_logger(__name__, log_level="DEBUG")
# log all processes
logger.debug("thing_to_log", main_process_only=False)
# log all processes in order
logger.debug("thing_to_log", main_process_only=False, in_order=True)

# Find avaialbel batch size to avoid cuda out-of-memory

In [ ]:
def training_function(args):
    accelerator = Accelerator()

    @find_executable_batch_size(starting_batch_size=args.batch_size)
    def inner_training_loop(batch_size):
        nonlocal accelerator # Ensure they can be used in our context
        accelerator.free_memory() # Free all lingering references
        model = get_model()
        model.to(accelerator.device)
        optimizer = get_optimizer()
        train_dataloader, eval_dataloader = get_dataloaders(accelerator, batch_size)
        lr_scheduler = get_scheduler(
            optimizer, 
            num_training_steps=len(train_dataloader)*num_epochs
        )
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
            model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
        )
        train(model, optimizer, train_dataloader, lr_scheduler)
        validate(model, eval_dataloader)
    inner_training_loop()